# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [97]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import csv

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [65]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

591

In [66]:
#print cities for sanity check
#print(cities)

['grand river south east', 'butaritari', 'pochutla', 'luanda', 'maryville', 'privokzalnyy', 'jamestown', 'hermanus', 'vaini', 'airai', 'mys shmidta', 'taolanaro', 'rikitea', 'dikson', 'ribeira grande', 'atuona', 'nikolskoye', 'georgetown', 'cockburn town', 'klaksvik', 'cape town', 'clyde river', 'yagodnoye', 'iskateley', 'naze', 'punta arenas', 'qaanaaq', 'ushuaia', 'arraial do cabo', 'tiznit', 'isla mujeres', 'carutapera', 'alice springs', 'geraldton', 'bubaque', 'cherskiy', 'fare', 'necochea', 'tautira', 'kahului', 'saldanha', 'warqla', 'yefimovskiy', 'pangody', 'sola', 'eureka', 'komsomolets', 'la serena', 'salalah', 'norman wells', 'masunga', 'mega', 'new norfolk', 'thompson', 'hilo', 'chokurdakh', 'mataura', 'belushya guba', 'tuktoyaktuk', 'port alfred', 'portland', 'puerto escondido', 'kapaa', 'albany', 'tessalit', 'mitu', 'ust-koksa', 'san quintin', 'longyearbyen', 'cabo san lucas', 'barrow', 'sheridan', 'bandar', 'leh', 'hithadhoo', 'los llanos de aridane', 'praia da vitoria', 

In [125]:
#create a sample response to look at how data is arranged in json
data_to_keep = ['city_name', 'lat', 'long', 'max_temp', 'cloudiness', 'wind_speed', 'humidity', 'country', 'date']
sample_req = requests.get(query_url+"kangaatsiaq").json()
data_to_keep.append(sample_req['name'])
data_to_keep.append(sample_req['coord']['lat'])
data_to_keep.append(sample_req['coord']['lon'])
data_to_keep.append(sample_req['main']['temp_max'])
data_to_keep.append(sample_req['weather'][0]['description'])
data_to_keep.append(sample_req['wind']['speed'])
data_to_keep.append(sample_req['main']['humidity'])
data_to_keep.append(sample_req['sys']['country'])
data_to_keep.append(sample_req['dt'])
print(data_to_keep)


['city_name', 'lat', 'long', 'max_temp', 'cloudiness', 'wind_speed', 'humidity', 'country', 'date', 'Kangaatsiaq', 68.3065, -53.4641, -3.51, 'overcast clouds', 2.54, 79, 'GL', 1619241026]


In [112]:
sample_req = requests.get(query_url+"kangaatsiaq").json()
print(sample_req)

{'coord': {'lon': -53.4641, 'lat': 68.3065}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': -3.51, 'feels_like': -7.23, 'temp_min': -3.51, 'temp_max': -3.51, 'pressure': 1019, 'humidity': 79, 'sea_level': 1019, 'grnd_level': 1019}, 'visibility': 10000, 'wind': {'speed': 2.54, 'deg': 88, 'gust': 2.52}, 'clouds': {'all': 100}, 'dt': 1619240850, 'sys': {'country': 'GL', 'sunrise': 1619247517, 'sunset': 1619309096}, 'timezone': -7200, 'id': 3422683, 'name': 'Kangaatsiaq', 'cod': 200}


In [126]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

query_url = f"{url}appid={weather_api_key}&units={units}&q="

#loop set up
data_to_keep = ['city_name', 'lat', 'long', 'max_temp', 'cloudiness', 'wind_speed', 'humidity', 'country', 'date']

for city in cities:
    try:
        keep_response = requests.get(query_url + city).json()
        sample_req = requests.get(query_url+"kangaatsiaq").json()
        data_to_keep.append(keep_response['name'])
        data_to_keep.append(keep_response['coord']['lat'])
        data_to_keep.append(keep_response['coord']['lon'])
        data_to_keep.append(keep_response['main']['temp_max'])
        data_to_keep.append(keep_response['weather'][0]['description'])
        data_to_keep.append(keep_response['wind']['speed'])
        data_to_keep.append(keep_response['main']['humidity'])
        data_to_keep.append(keep_response['sys']['country'])
        data_to_keep.append(keep_response['dt'])
        print(f'processing record for: {city}')
    
    except KeyError:
        print('records missing for {city} skipping...')

print('done')
print(data_to_keep)

47.8167, 10.9, 0.56, 'clear sky', 1.54, 80, 'DE', 1619241776, 'Yabrūd', 33.9674, 36.6574, 22.22, 'clear sky', 5.14, 21, 'SY', 1619241777, 'Flin Flon', 54.7682, -101.865, -5.59, 'scattered clouds', 0.75, 65, 'CA', 1619241778, 'Montepuez', -13.1256, 38.9997, 20.61, 'overcast clouds', 3.56, 87, 'MZ', 1619241778, 'Paamiut', 61.994, -49.6678, -0.09, 'clear sky', 1.69, 75, 'GL', 1619241779, 'Malanje', -9.5402, 16.341, 17.16, 'light rain', 1.03, 97, 'AO', 1619241779, 'Pahrump', 36.2083, -115.9839, 20.56, 'clear sky', 1.34, 17, 'US', 1619241780, 'Vanimo', -2.6741, 141.3028, 27.38, 'overcast clouds', 2.09, 83, 'PG', 1619241781, 'Mitsamiouli', -11.3847, 43.2844, 27, 'light rain', 4.12, 83, 'KM', 1619241781, 'Great Yarmouth', 52.6083, 1.7305, 5, 'overcast clouds', 1.03, 87, 'GB', 1619241782, 'Piaçabuçu', -10.4056, -36.4344, 24.88, 'few clouds', 1.83, 86, 'BR', 1619241782, 'Hauterive', 46.0901, 3.4459, 8, 'clear sky', 3.6, 66, 'FR', 1619241783, 'Muli', 2.9167, 73.5667, 28.66, 'few clouds', 3.99, 6

In [81]:
print(city_data)
output_data_file = "output_data/cities.csv"
city_dataframe = pd.DataFrame(city_data) 

# saving the dataframe 
city_dataframe.to_csv('cities.csv')

'speed': 1.54, 'deg': 220}, 'clouds': {'all': 20}, 'dt': 1619230531, 'sys': {'type': 1, 'id': 9333, 'country': 'ID', 'sunrise': 1619216498, 'sunset': 1619259158}, 'timezone': 28800, 'id': 1645528, 'name': 'Denpasar', 'cod': 200}, {'coord': {'lon': 148.0167, 'lat': -33.3833}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 20, 'feels_like': 19.02, 'temp_min': 20, 'temp_max': 20, 'pressure': 1023, 'humidity': 37}, 'visibility': 10000, 'wind': {'speed': 2.92, 'deg': 237, 'gust': 3.54}, 'clouds': {'all': 88}, 'dt': 1619230566, 'sys': {'type': 3, 'id': 20556, 'country': 'AU', 'sunrise': 1619210175, 'sunset': 1619249738}, 'timezone': 36000, 'id': 2166368, 'name': 'Forbes', 'cod': 200}, {'coord': {'lon': -77.6139, 'lat': -11.0964}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 19.36, 'feels_like': 19.65, 'temp_min': 19.36, 'temp_max': 1

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [105]:
#weather checks
#url = "http://api.openweathermap.org/data/2.5/weather?"
#units = "metric"

#query_url = f"{url}appid={weather_api_key}&units={units}&q="

#loop set up
#lat = []
#lng = []
#max_temp = []
#humidity = []
#cloudiness = []
#windspeed = []
#country = []
#date = []
#city_names = []
#for city in cities:
    #try:
        #response = requests.get(query_url + city).json()
        #city_names.append(city)
        #lat.append(response['coord']['lat'])
        #lng.append(response['coord']['lng'])
        #max_temp.append(response['main']['temp_max'])
        #cloudiness.append(response['weather']['description'])
        #windspeed.append(response['wind']['speed'])
        #country.append(response['sys']['country'])
        #date.append(response['dt'])
        #print(f'processing record for: {city}')
    
    #except KeyError:
        #print('records missing for {city} skipping...')

#print('done')

In [106]:
#df creation
#weather_dict = {
    #"city": city_names,
    #"latitude": lat,
    #"longitude": lng,
    #"max temperature": max_temp,
    #"humidity": humidity,
    #"cloudiness": cloudiness,
    #"windspeed": windspeed,
    #"country": country,
    #"date": date
#}
#print(len(city_names))
#print(len(lat))
#print(len(lng))
#print(len(max_temp))
#print(len(humidity))
#print(len(cloudiness))
#print(len(windspeed))
#print(len(country))
#print(len(date))

#weather_data = pd.DataFrame(weather_dict)
#weather_data.head()

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [104]:
print(city_dataframe.dtypes)
city_dataframe.head()


cod            object
message        object
coord          object
weather        object
base           object
main           object
visibility    float64
wind           object
rain           object
clouds         object
dt            float64
sys            object
timezone      float64
id            float64
name           object
snow           object
dtype: object


,cod,message,coord,weather,base,main,visibility,wind,rain,clouds,dt,sys,timezone,id,name,snow
0,404,city not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200,NaN,"{'lon': 172.7902, 'lat': 3.0707}","[{'id': 501, 'main': 'Rain', 'description': 'm...",stations,"{'temp': 27.43, 'feels_like': 30.48, 'temp_min...",10000.0,"{'speed': 2.96, 'deg': 85, 'gust': 3.24}",{'1h': 1.75},{'all': 100},1.619230e+09,"{'country': 'KI', 'sunrise': 1619202040, 'suns...",43200.0,2110227.0,Butaritari,NaN
2,200,NaN,"{'lon': -96.4661, 'lat': 15.7432}","[{'id': 804, 'main': 'Clouds', 'description': ...",stations,"{'temp': 27.78, 'feels_like': 30.75, 'temp_min...",10000.0,"{'speed': 1.79, 'deg': 312, 'gust': 2.68}",NaN,{'all': 100},1.619230e+09,"{'type': 3, 'id': 2005410, 'country': 'MX', 's...",-18000.0,3517970.0,Pochutla,NaN
3,200,NaN,"{'lon': 13.2343, 'lat': -8.8368}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 25, 'feels_like': 25.85, 'temp_min': ...",10000.0,"{'speed': 3.09, 'deg': 70}",NaN,{'all': 40},1.619230e+09,"{'type': 1, 'id': 2173, 'country': 'AO', 'sunr...",3600.0,2240449.0,Luanda,NaN
4,200,NaN,"{'lon': -83.9705, 'lat': 35.7565}","[{'id': 803, 'main': 'Clouds', 'description': ...",stations,"{'temp': 9.16, 'feels_like': 8.16, 'temp_min':...",10000.0,"{'speed': 2.06, 'deg': 90}",NaN,{'all': 75},1.619230e+09,"{'type': 1, 'id': 6098, 'country': 'US', 'sunr...",-14400.0,4639848.0,Maryville,NaN


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [69]:
#  Get the indices of cities that have humidity over 100%.


In [70]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression